### Writers

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ipqa-research/ugropy/blob/main/docs/source/tutorial/writers.ipynb)

#### Clapeyron (https://github.com/ClapeyronThermo/Clapeyron.jl)
`ugropy` provides a writers module for constructing input files for various
thermodynamic libraries.

To utilize this function, you must import the module as follows:

In [1]:
try:
  import google.colab
  %pip install -q ugropy
except ImportError:
  pass

In [2]:
from ugropy import Groups, writers

To utilize the function, you need to provide a list of dictionaries for the
functional groups of UNIFAC and PSRK, where each dictionary contains the
functional groups of the molecules.

If the user wishes to write critical properties .csv files, they must provide a
list of Joback objects. Let's illustrate this with a simple example:

In [3]:
names = ["limonene", "adrenaline", "Trinitrotoluene"]

grps = [Groups(n) for n in names]

# Write the csv files into a database directory
writers.to_clapeyron(
    molecules_names=names,
    unifac_groups=[g.unifac.subgroups for g in grps],
    psrk_groups=[g.psrk.subgroups for g in grps],
    dortmund_groups=[g.dortmund.subgroups for g in grps],
    property_estimator=[g.joback for g in grps],
    path="database"
)

In the example provided, we create a Groups object to obtain all the
information of the molecules. Then, we use list comprehension to create the
lists for the to_clapeyron function.

The molecules_name argument in this case receives the names used to create the
Groups objects, but it can be different if desired. These names will be set as
the molecule names in the .csv files.

You can omit certain arguments if desired:

- If you omit the psrk_groups argument: the PSRK_groups.csv file will not be created.
- If you omit the dortmund_groups argument: the UNIFAC_groups.csv file will not be created.
- If you omit the unifac_groups argument: the ogUNIFAC_groups.csv file will not be created.
- If you omit the joback_objects argument: the critical.csv file will not be created.

#### Thermo (https://github.com/CalebBell/thermo)

`ugropy` also provides a translator of its subgroups dictionaries to the
`Thermo` library dictionaries.


In [4]:
from thermo.unifac import UFIP, UFSG, UNIFAC

from ugropy import unifac, writers


names = ["hexane", "ethanol"]

grps = [unifac.get_groups(n) for n in names]

thermo_groups = [writers.to_thermo(g.subgroups, unifac) for g in grps]

print(thermo_groups[0])
print(thermo_groups[1])

{1: 2, 2: 4}
{1: 1, 2: 1, 14: 1}


In [5]:
GE = UNIFAC.from_subgroups(
    chemgroups=thermo_groups,
    T=60+273.15,
    xs=[0.5, 0.5],
    version=0,
    interaction_data=UFIP,
    subgroups=UFSG
)

GE.gammas()

[1.829336850520012, 1.5690194318563724]

#### yaeos (https://github.com/ipqa-research/yaeos)

`ugropy` also provides a translator of its subgroups dictionaries to the
`yaeos` Fortran source code.

In [6]:
from ugropy import unifac, writers

names = ["hexane", "ethanol", "limonene"]

grps = [unifac.get_groups(n).subgroups for n in names]

fortran_code = writers.to_yaeos(grps, unifac)

print(fortran_code)

use yaeos__models_ge_group_contribution_unifac, only: Groups

type(Groups) :: molecules(3)

molecules(1)%groups_ids = [1, 2]
molecules(1)%number_of_groups = [2, 4]

molecules(2)%groups_ids = [1, 2, 14]
molecules(2)%number_of_groups = [1, 1, 1]

molecules(3)%groups_ids = [1, 2, 3, 7, 8]
molecules(3)%number_of_groups = [2, 3, 1, 1, 1]


